# Train
### 1. 데이터 처음 학습하기

1. `python src/KoBART/train.py` 실행


2. 입력값   
    - **train_file**   
    `train 데이터 주소` 입력
    - **test_file**   
    `test 데이터 주소` 입력
    - **mode**  
    `train` 입력
    - **batch_size**   
    `batch size` (ex.16) 입력  
    auto_scale_batch_size='power' 등을 설정하더라도 임의의 값을 입력해야 함
    - **num_workers**  
    `num_workers` (ex.10) 입력   
    가상 cpu 코어 개수 : 72  
    물리적 코어 개수 = 가상 cpu 코어 개수의 절반 = 36   
    적절한 num workers = 물리적 코어 개수의 절반 = 18  
    num workers 18로 진행시 -> Bus error (out of shared memory) -> 도커의 shared memory 변경하면 오류 해결 가능할 것으로 예상
    - **gradient_clip_val**   
    `default 값`인 1.0 입력    
    - **gpus**   
    `제공되는 gpu의 개수` (ex.2) 인 입력
    - **accelerator**   
    multi gpu 사용이므로`ddp` 입력  
    - **max_epochs**  
    `최대 epoch 수` (ex.50) 입력
    - **default_root_dir**  
    `학습 결과를 저장할 폴더 경로` 입력
    
    
3. 저장된 학습 결과 확인  
    **default_root_dir** 폴더 안에 생성되는 폴더 및 파일들
    - **kobart_summary-model_chp**   
        : `epoch=xx-val_loss=x.xxx.ckpt` 의 형태로 체크포인트 파일 생성
    - **tb_logs > default**   
        : `version_x` 의 형태로 버전 폴더 생성   
        : 버전 폴더 안 `events.out.tfevents.xxxxxxxxxxxx`, `hparams.yaml` 의 형태로 이벤트 파일과 하이퍼파라미터 파일 생성
    - **kobart_summary-last.ckpt**   
        : 가장 마지막 체크포인트 파일 생성

아래 accelerator 옵션을
mac os -> mps
cuda (nvidia) --> gpu
그래픽 카드 없으면 --> cpu

In [11]:
!python src/KoBART/train.py \
--train_file='rsc/1.Training/news_summaries.csv' \
--test_file='rsc/2.Validation/news_summaries.csv' \
--mode='train' \
--batch_size=8 \
--num_workers=8 \
--gradient_clip_val=1.0 \
--accelerator='mps' \
--max_epochs=50 \
--default_root_dir='rsc/By_domain_ckpt/aihub'

/Users/simmin-a/Desktop/project/newj/.venv/lib/python3.9/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/Users/simmin-a/Desktop/project/newj/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
2024-08-18 23:36:40.746 | INFO     | __main__:<module>:83 - Namespace(train_file='rsc/1.Training/news_summaries.csv', test_file='rsc/2.Validation/news_summaries.csv', batch_size=4, checkpoint='checkpoint', max_len=512, max_epochs=3, lr=3e-05, ac

### 2. 데이터 이어 학습하기 (체크포인트 이어 학습)

1. `python src/KoBART/train.py` 실행


2. 입력값
    - **checkpoint_path**    
    학습한 모델의 체크포인트 중 val_loss 가 가장 낮은 `체크포인트 파일 경로` 입력
    - **hparams_file**   
    학습한 모델의 `하이퍼파라미터 파일 경로` 입력
    - 나머지는 위와 동일
    
    
3. 저장된 학습 결과 확인   
    위와 동일 

In [ ]:
!python src/KoBART/train.py \
--train_file='rsc/1.Training/news_summaries.csv' \
--test_file='rsc/2.Validation/news_summaries.csv' \
--checkpoint_path='checkpoint/model_chp/epoch=01-val_loss=1.660.ckpt' \
--hparams_file='rsc/By_domain_ckpt/dacon신문기사/tb_logs/default/version_1/hparams.yaml' \
--mode='train' \
--batch_size=8 \
--num_workers=8 \
--gradient_clip_val=1.0 \
--accelerator=mps \
--max_epochs=50 \
--default_root_dir='rsc/By_domain_ckpt/aihub'

# Test
### 학습한 모델로 테스트하기 & rouge 값 출력하기

1. `python src/KoBART/train.py` 실행


2. 입력값   
    - **test_file**   
    `test 데이터 주소` 입력
    - **mode**  
    `test` 입력
    - **checkpoint_path**    
    학습한 모델의 체크포인트 중 val_loss 가 가장 낮은 `체크포인트 파일 경로` 입력
    - **hparams_file**   
    학습한 모델의 `하이퍼파라미터 파일 경로` 입력
    - **default_root_dir**  
    `test 결과를 저장할 폴더 경로` 입력
    - 나머지는 위와 동일
    
    
3. 저장된 테스트 결과 확인  
    **default_root_dir** 폴더 안에 생성되는 폴더 및 파일들
    - **rouge_score.csv**     
        : csv 형태로 rouge score가 출력된 파일 생성
    - **tb_logs > default**   
        : `version_x` 의 형태로 버전 폴더 생성   
        : 버전 폴더 안 `events.out.tfevents.xxxxxxxxxxxx`, `hparams.yaml` 의 형태로 이벤트 파일과 하이퍼파라미터 파일 생성
    - 이 외 파일은 생성되지 않음

In [23]:
!python src/KoBART/train.py \
--train_file='rsc/1.Training/news_summaries.csv' \
--test_file='rsc/2.Validation/news_summaries.csv' \
--mode='test' \
--checkpoint_path='checkpoint/last.ckpt' \
--batch_size=8 \
--num_workers=8 \
--accelerator=mps \
--default_root_dir='rsc/By_hpram_ckpt/batch_4/not_lr_finder/test_dacon'

INFO:root:Namespace(accelerator='ddp', accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=16, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=True, checkpoint_path='rsc/KoBART-Dacon/batch_4/not_lr_finder/num_workers4/kobart_summary-model_chp/epoch=02-val_loss=1.358.ckpt', default_root_dir='rsc/KoBART-test', deterministic=False, distributed_backend=None, enable_pl_optimizer=True, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=2, gradient_clip_val=0, hparams_file='rsc/KoBART-Dacon/batch_4/not_lr_finder/num_workers4/tb_logs/default/version_0/hparams.yaml', limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=1000, max_len=512, max_steps=None, min_epochs=1, min_steps=None, mode='test', model_path=None, move_metrics_to_cpu=False, num_nodes=1, num_p

In [2]:
!python src/KoBART/download_binary.py

zsh:1: command not found: python
